In [1]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import welch
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
save_path = 'D:\EECE499\Features\GSRFeatures.xlsx'
#save_path = '..\..\GSRFeatures.xlsx'

In [4]:
sample_rate = 128

In [5]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=6):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
max_length = 17215

In [9]:
yup = np.array(list(range(1, 17216)))

In [11]:
yup

array([    1,     2,     3, ..., 17213, 17214, 17215])

In [23]:
17216/2460

6.998373983739837

In [45]:
window = 2460
x = [2460, 4920, 7380, 9840, 12300, 14760]
y = [1230, '-', 3690, '-', 7380, '-', 9840, '-', 12300]

In [46]:
window_intervals_even = np.array([1, 2, 3, 4, 5, 6])*window
window_intervals_odd = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])*window

In [47]:
window_intervals_even

array([ 2460,  4920,  7380,  9840, 12300, 14760])

In [52]:
window_intervals_odd = window_intervals_odd.astype(int)
window_intervals_odd

array([ 1230,  3690,  6150,  8610, 11070, 13530, 15990])

In [54]:
for i in range(1, 6):
    print(yup[window_intervals_even[i - 1]:window_intervals_even[i]].shape[0] == 2460)
    
for i in range(1, 7):
    print(yup[window_intervals_odd[i - 1]:window_intervals_odd[i]].shape[0] == 2460)

True
True
True
True
True
True
True
True
True
True
True


In [65]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))
    
    for clip_id in range(1, 37):
        #print('Clip ID: ' + str(clip_id))
        
        #data_path = 'D:/EECE499/Raw/MyECGFunc/ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'    
        data_path = './../../ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'
        
        #presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        if os.path.isfile(data_path):
            
            signal = scipy.io.loadmat(data_path)
            resistance = signal['Data_GSR'][:,4]
            time_stamp = signal['Data_GSR'][:,0]
            
            resistance = np.array(resistance)
            if (max_length > resistance.shape[0]):
                resistance = np.pad(resistance, (0, max_length - resistance.shape[0]), 'constant')
            else:
                resistance = resistance[:17215]
            #print(resistance.shape)
            
            

User ID: 1
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 2
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 3
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(172

(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 26
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 27
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 28
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(

(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 51
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 52
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
User ID: 53
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(17215,)
(